In [1]:
from bs4 import BeautifulSoup as bs
import requests
import os
import pandas as pd
from IPython.display import clear_output

In [2]:
tumiller = ['istanbul', 'ankara', 'izmir', 'adana', 'adiyaman', 'afyonkarahisar', 'agri', 'aksaray', 'amasya', 'antalya', 'ardahan', 'artvin', 'aydin', 'balikesir', 'bartin', 'batman', 
         'bayburt', 'bilecik', 'bingol', 'bitlis', 'bolu', 'burdur', 'bursa', 'canakkale', 'cankiri', 'corum', 'denizli', 'diyarbakir', 'duzce', 'edirne', 'elazig', 'erzincan', 'erzurum', 
         'eskisehir', 'gaziantep', 'giresun', 'gumushane', 'hakkari', 'hatay', 'igdir', 'isparta', 'kahramanmaras', 'karabuk', 'karaman', 'kars', 'kastamonu', 'kayseri', 'kirikkale', 
         'kirklareli', 'kirsehir', 'kilis', 'kocaeli', 'konya', 'kutahya', 'malatya', 'manisa', 'mardin', 'mersin', 'mugla', 'mus', 'nevsehir', 'nigde', 'ordu', 'osmaniye', 'rize', 'sakarya', 
         'samsun', 'siirt', 'sinop', 'sivas', 'sanliurfa', 'sirnak', 'tekirdag', 'tokat', 'trabzon', 'tunceli', 'usak', 'van', 'yalova', 'yozgat', 'zonguldak']

In [3]:
iller = []
sayfalar = []
for il in tumiller:
    url="https://www.zingat.com/" + il +"-satilik-daire?page="
    iller.append(url)
z = 1
for i in range(0,len(iller)):
    clear_output(wait=True)
    print(z)
    illistesi = requests.get(iller[i]).text
    soup = bs(illistesi, 'html.parser')
    try:
        pag = soup.find("nav", "zng-pagination")['data-total']
        for j in range(2, int(pag)):
            sayfa = iller[i] + str(j)
            sayfalar.append(sayfa)
            z = z + 1
    except:
        continue
sayfalar.extend(iller)

908


In [4]:
sayfaurl = []
satisurl = []
k = 1
m = 1
for sayfa in sayfalar:
    clear_output(wait=True)
    print(m)
    req = requests.get(sayfa)
    soup = bs(req.text, "html.parser")
    sayfaurl.append(soup.select(".zl-card-inner"))
    m = m + 1
for page in sayfaurl:
    for link in page:
        satisurl.append("https://www.zingat.com" + link.get("href"))

988


In [5]:
ozellikler = []
fiyatlar = []
mahalle = []
ilce =[]
il = []
gun = []
ay = []
yil = []
n = 1

for url in satisurl:
    try:
        clear_output(wait=True) 
        print(n)
        req = requests.get(url)
        soup = bs(req.text, "html.parser")
        ozellikler.append([tag.text.strip() for tag in soup.find_all(["strong", "span"], {"class": "col-md-6"})])
        fiyatlar.append(soup.find("strong", itemprop = "price").text.replace("TL", "").replace(".","").replace(" ",""))
        mahalle.append(soup.find("div", "detail-location-path__map").find("h2").text.replace("\n","").replace(" ", "").split(",")[0])
        ilce.append(soup.find("div", "detail-location-path__map").find("h2").text.replace("\n","").replace(" ", "").split(",")[1])
        il.append(soup.find("div", "detail-location-path__map").find("h2").text.replace("\n","").replace(" ", "").split(",")[2])
        gun.append(soup.find("div", "detail-info not-printable").find_all("div")[3].find("strong").text.split(" ")[0])
        ay.append(soup.find("div", "detail-info not-printable").find_all("div")[3].find("strong").text.split(" ")[1])
        yil.append(soup.find("div", "detail-info not-printable").find_all("div")[3].find("strong").text.split(" ")[2])
        n = n + 1
    except:
        continue
    
ozelliklerYeni = []
for ozellik in ozellikler:
    ozelliklerYeni.append(dict(zip(ozellik[::2], ozellik[1::2])))


20471


In [316]:
df = pd.DataFrame(ozelliklerYeni)
df['Fiyat'] = pd.Series(fiyatlar)
df['İl'] = pd.Series(il)
df['İlçe'] = pd.Series(ilce)
df['Mahalle'] = pd.Series(mahalle)
df['İlan Gün'] = pd.Series(gun)
df['İlan Ay'] = pd.Series(ay)
df["İlan Yıl"] = pd.Series(yil)

df.drop(['Takasa Uygun','Aidat','Görüntülü Ev Gezintisi\n                                                                ?',
         'Kira Getirisi','Yapının Durumu','Ada','Kimden','Konut Şekli','Hayvan Dostu Patili Evler','Tapu Durumu','Parsel','Listing No',
       'Gross m²', 'Number of Rooms', 'Number of Bathrooms','Mobilya Durumu',
       'Number of Floors of the Building', 'Heating Type', 'Floor','Bina Yaşı',
       'Age of the Building', 'Usage Status', 'Status of the Property',
       'Maintenance Fee', 'Rental Income', 'Suitable for Bank Loan','Kullanım Durumu','Mahalle','İlan Gün', 'İlan Ay', 'İlan Yıl',
       'House Type','Layout',
       'Video Home Tour\n                                                                ?',
       'Furnishing Status', 'From Whom', 'Title Deed Status',
       'Pet Friendly Paws Houses', 'Unit Exchange','Parcelled'], axis=1, inplace=True)

df = df['Oda Sayısı'].str.rsplit('+', 0, expand=True).join(df)
df=df.rename(columns = {0:'İlan Oda Sayısı'})
df=df.rename(columns = {1:'İlan Salon Sayısı'})
del df['Oda Sayısı']
df[['Brüt m²']] = df[['Brüt m²']].replace('m²', '', regex=True)
df

,İlan Oda Sayısı,İlan Salon Sayısı,İlan no,Net m²,Brüt m²,Banyo Sayısı,Binadaki Kat Sayısı,Isıtma Tipi,Bulunduğu Kat,Krediye Uygun,Fiyat,İl,İlçe
0,1,1,4291369,60,70,1,5,Merkezi Sistem,3,Evet,3575000,İstanbul,Sarıyer
1,2,1,4307880,68,90,1,5,Kombi (Doğalgaz),Bahçe katı,Evet,2499000,İstanbul,Beylikdüzü
2,1,1,4317006,55,70,1,10-20 arası,Kombi (Doğalgaz),1,Hayır,890000,İstanbul,Esenyurt
3,2,1,4316997,95,105,1,4,Kombi (Doğalgaz),1,Evet,679000,İstanbul,Beylikdüzü
4,2,1,4316979,100,110,1,5,Kombi (Doğalgaz),Yüksek Giriş,Evet,470000,İstanbul,Beylikdüzü
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20490,2,1,4135819,120,125,2,6,Kombi (Doğalgaz),6,Hayır,NaN,NaN,NaN
20491,2,1,4230651,100,110,1,5,Kombi (Doğalgaz),2,Hayır,NaN,NaN,NaN
20492,3,1,4254324,135,137,2,5,Kombi (Doğalgaz),2,Evet,NaN,NaN,NaN
20493,3,1,4254326,135,137,2,5,Kombi (Doğalgaz),3,Evet,NaN,NaN,NaN


In [317]:
df["Isıtma Tipi"].value_counts()

Kombi (Doğalgaz)                     10558
Klima                                 3101
Merkezi Sistem                        1305
Yerden Isıtma                         1000
Merkezi Sistem (Isı Payı Ölçer)        947
Kat Kaloriferi                         351
Kalorifer (Doğalgaz)                   320
Soba (Kömür)                           303
Yok                                    254
Soba (Doğalgaz)                        109
Güneş Enerjisi                          77
Jeotermal                               69
Kalorifer (Kömür)                       56
Combi Boiler (Natural Gas)              31
Kombi (Elektrikli)                      28
Air Conditioner                         18
Fancoil                                 15
Kalorifer (Akaryakıt)                    9
Central System (Heat Share Meter)        5
Floor Heating                            5
Independent Boiler                       3
Central System                           2
Central Heating (Natural Gas)            2
Geothermal 

In [318]:
df["Isıtma Tipi"] = df["Isıtma Tipi"].fillna('0')
df.loc[df["Isıtma Tipi"] == "Yok" , "Isıtma Tipi"] = "0"
df.loc[df["Isıtma Tipi"] == "None" , "Isıtma Tipi"] = "0"
df.loc[df["Isıtma Tipi"] == "Soba (Doğalgaz)" , "Isıtma Tipi"] = "0"
df.loc[df["Isıtma Tipi"] == "Soba (Kömür)" , "Isıtma Tipi"] = "0"

df.loc[df["Isıtma Tipi"] == "Kat Kaloriferi" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Merkezi Sistem (Isı Payı Ölçer)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Kombi (Doğalgaz)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Kalorifer (Doğalgaz)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Kombi (Elektrikli)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Kalorifer (Akaryakıt)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Kalorifer (Kömür)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "-" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Merkezi Sistem" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Central System" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Central Heating (Natural Gas)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Central System (Heat Share Meter)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Fancoil" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Floor Heating" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Independent Boiler" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Combi Boiler (Natural Gas)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Stove (Natural Gas)" , "Isıtma Tipi"] = "1"
df.loc[df["Isıtma Tipi"] == "Stove (Coal)" , "Isıtma Tipi"] = "1"

df.loc[df["Isıtma Tipi"] == "Klima" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Air Conditioner" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Yerden Isıtma" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Jeotermal" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Geothermal" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Güneş Enerjisi" , "Isıtma Tipi"] = "2"
df.loc[df["Isıtma Tipi"] == "Solar Power" , "Isıtma Tipi"] = "2"

In [319]:
df["Isıtma Tipi"].value_counts()

1    13639
2     4267
0     2589
Name: Isıtma Tipi, dtype: int64

In [320]:
df = df[df['İlan no'].notna()]
df = df[df['Fiyat'].notna()]
#df = df[df['İlan Gün'].notna()]
#df = df[df['İlan Ay'].notna()]
#df = df[df['İlan Yıl'].notna()]
df = df[df['Bulunduğu Kat'].notna()]
#df = df[df['Bina Yaşı'].notna()]
df = df[df['İlan Oda Sayısı'].notna()]
df = df[df['İlan Salon Sayısı'].notna()]
df = df[df['Binadaki Kat Sayısı'].notna()]

df['Brüt m²'] = pd.to_numeric(df['Brüt m²'])
df.drop(df[(df['Brüt m²']>5000)].index, inplace=True)

In [321]:
df["İlan Salon Sayısı"].value_counts()

1             17138
2               448
0                64
0 (Stüdyo)       42
3                26
4                 3
Name: İlan Salon Sayısı, dtype: int64

In [322]:
df.loc[df["İlan Salon Sayısı"] == "0 (Stüdyo)" , "İlan Salon Sayısı"] = "0"

In [323]:
df["İlan Salon Sayısı"].value_counts()

1    17138
2      448
0      106
3       26
4        3
Name: İlan Salon Sayısı, dtype: int64

In [324]:
df["Krediye Uygun"].value_counts()

Evet     15712
Hayır     1951
Yes         58
Name: Krediye Uygun, dtype: int64

In [325]:
df.loc[df["Krediye Uygun"] == "Evet" , "Krediye Uygun"] = "1"
df.loc[df["Krediye Uygun"] == "Yes" , "Krediye Uygun"] = "1"
df.loc[df["Krediye Uygun"] == "Hayır" , "Krediye Uygun"] = "0"
df.loc[df["Krediye Uygun"] == "No" , "Krediye Uygun"] = "0"

In [326]:
df["Krediye Uygun"].value_counts()

1    15770
0     1951
Name: Krediye Uygun, dtype: int64

In [328]:
df.reset_index(drop=True, inplace=True)
df

,İlan Oda Sayısı,İlan Salon Sayısı,İlan no,Net m²,Brüt m²,Banyo Sayısı,Binadaki Kat Sayısı,Isıtma Tipi,Bulunduğu Kat,Krediye Uygun,Fiyat,İl,İlçe
0,1,1,4291369,60,70,1,5,1,3,1,3575000,İstanbul,Sarıyer
1,2,1,4307880,68,90,1,5,1,Bahçe katı,1,2499000,İstanbul,Beylikdüzü
2,1,1,4317006,55,70,1,10-20 arası,1,1,0,890000,İstanbul,Esenyurt
3,2,1,4316997,95,105,1,4,1,1,1,679000,İstanbul,Beylikdüzü
4,2,1,4316979,100,110,1,5,1,Yüksek Giriş,1,470000,İstanbul,Beylikdüzü
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17716,1,1,3921291,60,65,1,5,1,Çatı Katı,1,750000,Zonguldak,ZonguldakMerkez
17717,4,2,3719036,170,180,2,2,1,Kot 1,1,650000,Zonguldak,Alaplı
17718,2,1,4106599,80,90,1,5,1,4,0,1000000,Zonguldak,Alaplı
17719,3,2,4152965,220,300,2,10,1,10,1,1050000,Zonguldak,Alaplı


In [329]:
df[['Fiyat']] = df[['Fiyat']].replace('TRY','', regex=True)
df["Fiyat"]=df["Fiyat"].astype(float)
df

,İlan Oda Sayısı,İlan Salon Sayısı,İlan no,Net m²,Brüt m²,Banyo Sayısı,Binadaki Kat Sayısı,Isıtma Tipi,Bulunduğu Kat,Krediye Uygun,Fiyat,İl,İlçe
0,1,1,4291369,60,70,1,5,1,3,1,3575000.0,İstanbul,Sarıyer
1,2,1,4307880,68,90,1,5,1,Bahçe katı,1,2499000.0,İstanbul,Beylikdüzü
2,1,1,4317006,55,70,1,10-20 arası,1,1,0,890000.0,İstanbul,Esenyurt
3,2,1,4316997,95,105,1,4,1,1,1,679000.0,İstanbul,Beylikdüzü
4,2,1,4316979,100,110,1,5,1,Yüksek Giriş,1,470000.0,İstanbul,Beylikdüzü
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17716,1,1,3921291,60,65,1,5,1,Çatı Katı,1,750000.0,Zonguldak,ZonguldakMerkez
17717,4,2,3719036,170,180,2,2,1,Kot 1,1,650000.0,Zonguldak,Alaplı
17718,2,1,4106599,80,90,1,5,1,4,0,1000000.0,Zonguldak,Alaplı
17719,3,2,4152965,220,300,2,10,1,10,1,1050000.0,Zonguldak,Alaplı


In [330]:
old_cols = df.columns.values 
new_cols = ['İlan no','İlan Oda Sayısı', 'İlan Salon Sayısı',
       'Net m²', 'Brüt m²', 'Banyo Sayısı', 'Binadaki Kat Sayısı',
       'Isıtma Tipi', 'Bulunduğu Kat', 'Krediye Uygun',
       'İl', 'İlçe','Fiyat']
df = df.reindex(columns=new_cols)

In [331]:
df.to_csv("filtreVeriler.csv")

In [332]:
df

,İlan no,İlan Oda Sayısı,İlan Salon Sayısı,Net m²,Brüt m²,Banyo Sayısı,Binadaki Kat Sayısı,Isıtma Tipi,Bulunduğu Kat,Krediye Uygun,İl,İlçe,Fiyat
0,4291369,1,1,60,70,1,5,1,3,1,İstanbul,Sarıyer,3575000.0
1,4307880,2,1,68,90,1,5,1,Bahçe katı,1,İstanbul,Beylikdüzü,2499000.0
2,4317006,1,1,55,70,1,10-20 arası,1,1,0,İstanbul,Esenyurt,890000.0
3,4316997,2,1,95,105,1,4,1,1,1,İstanbul,Beylikdüzü,679000.0
4,4316979,2,1,100,110,1,5,1,Yüksek Giriş,1,İstanbul,Beylikdüzü,470000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17716,3921291,1,1,60,65,1,5,1,Çatı Katı,1,Zonguldak,ZonguldakMerkez,750000.0
17717,3719036,4,2,170,180,2,2,1,Kot 1,1,Zonguldak,Alaplı,650000.0
17718,4106599,2,1,80,90,1,5,1,4,0,Zonguldak,Alaplı,1000000.0
17719,4152965,3,2,220,300,2,10,1,10,1,Zonguldak,Alaplı,1050000.0


In [333]:
df.columns

Index(['İlan no', 'İlan Oda Sayısı', 'İlan Salon Sayısı', 'Net m²', 'Brüt m²',
       'Banyo Sayısı', 'Binadaki Kat Sayısı', 'Isıtma Tipi', 'Bulunduğu Kat',
       'Krediye Uygun', 'İl', 'İlçe', 'Fiyat'],
      dtype='object')

In [334]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17721 entries, 0 to 17720
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   İlan no              17721 non-null  object 
 1   İlan Oda Sayısı      17721 non-null  object 
 2   İlan Salon Sayısı    17721 non-null  object 
 3   Net m²               17721 non-null  object 
 4   Brüt m²              17721 non-null  int64  
 5   Banyo Sayısı         17721 non-null  object 
 6   Binadaki Kat Sayısı  17721 non-null  object 
 7   Isıtma Tipi          17721 non-null  object 
 8   Bulunduğu Kat        17721 non-null  object 
 9   Krediye Uygun        17721 non-null  object 
 10  İl                   17721 non-null  object 
 11  İlçe                 17721 non-null  object 
 12  Fiyat                17721 non-null  float64
dtypes: float64(1), int64(1), object(11)
memory usage: 1.8+ MB
